In [1]:
import numpy as np
from ase.io import read, write
import spglib
import os

import numpy as np
from ase.io import read
from calorine.calculators import CPUNEP
from calorine.tools import get_force_constants, relax_structure
from matplotlib import pyplot as plt
from pandas import DataFrame
from phonopy.units import THzToCm
from seekpath import get_explicit_k_path
import spglib
from ase.spacegroup.symmetrize import FixSymmetry
from ase.constraints import UnitCellFilter
from ase.optimize import BFGS


In [2]:
# parameters
for n in range(1,7):
    alat = 5.0
    tol = 0.1
    z_spacing = 3.0
    
    atoms_ideal = read('BaZrS3_cubic_2x2x2_with_modes.extxyz')
    atoms_ideal.symbols[atoms_ideal.symbols == 'O'] = 'S'
    
    new_cell = 2 * alat * np.eye(3)
    atoms_ideal.set_cell(new_cell, scale_atoms=True)
    atoms_ideal = atoms_ideal.repeat((1, 1, n))
    
    
    # build slab I
    atoms1 = atoms_ideal.copy()
    mask = atoms1.positions[:, 2] < n * alat + tol
    atoms1 = atoms1[mask]
    new_cell = np.diag([2 * alat, 2 * alat, n * 2 * alat + 2 * z_spacing])
    atoms1.set_cell(new_cell)
    
    # build slab II
    atoms2 = atoms1.copy()
    atoms2.translate([alat/2, alat/2, n * alat + z_spacing])
    
    
    # modes
    modes = ['Mx', 'My', 'Mz', 'Rx', 'Ry', 'Rz']
    for mode in modes:
        atoms1.set_array(mode + '1', atoms1.get_array(mode))
        atoms1.set_array(mode, None)
    
        atoms2.set_array(mode + '2', atoms2.get_array(mode))
        atoms2.set_array(mode, None)
    
    # build full
    atoms_full = atoms1.copy()
    atoms_full.extend(atoms2)
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms_full))
    write(f'tilt_structures/RP_{n}_000_000.in', atoms_full, format='aims')
    amp = 1


    # apply modes

    # tilt 2 
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += -amp * atoms.get_array('Mz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms), 'MzMz')
    #write(f'RP_{n}_00Mz_00Mz_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_00Mz_00Mz.in', traj, format='aims')

    # tilt 3
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Mz1')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms), "")
    #write(f'RP_{n}_00Mz_000_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_00Mz_000.in', traj, format='aims')
    
    
    # tilt 4
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += 0.5 * amp * atoms.get_array('Mz2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_00Mz1_00Mz2_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_00Mz1_00Mz2.in', traj, format='aims')
    
    # tilt 5
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += amp * atoms.get_array('Rz2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_00Rz_00Rz_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_00Rz_00Rz.in', traj, format='aims')
    
    # tilt 6
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rz1')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_00Rz_000_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_00Rz_000.in', traj, format='aims')
    
    # tilt 7
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += 0.5 * amp * atoms.get_array('Rz2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_00Rz1_00Rz2_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_00Rz1_00Rz2.in', traj, format='aims')
    
    # tilt 8
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_Rx00_0Ry0_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx00_0Ry0.in', traj, format='aims')
    
    # tilt 9
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_RxRy0_RxRy0_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_RxRy0_RxRy0.in', traj, format='aims')
   
    # tilt 10
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += 0.5 * amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += 0.5* amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_Rx1Ry20_Rx2Ry10_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')    
    write(f'tilt_structures/RP_{n}_Rx1Ry20_Rx2Ry10.in', traj, format='aims')    
    
    # tilt 11
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_Rx00_0-Ry0_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx00_0-Ry0.in', traj, format='aims')
    
    # tilt 12
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)
    
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_RxRy0_-Rx-Ry0_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_RxRy0_-Rx-Ry0.in', traj, format='aims')
   
#    # tilt 13
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += 0.5*amp * atoms.get_array('Ry1')
    atoms.positions += -0.5*amp * atoms.get_array('Rx2')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx1Ry20_-Rx2-Ry10.in', traj, format='aims')
    
#    # two tilts 
    # tilt 14
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Mz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Mz_0RyMz.in', traj, format='aims')

    
#    # tilt 15
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Rz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Rz_0RyRz.in', traj, format='aims')
#
    # tilt 16
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Mz2')
    traj.append(atoms)
#
    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Mz_0-RyMz.in', traj, format='aims')
#
#    # tilt 17
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Rz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Rz_0-RyRz.in', traj, format='aims')

#    # tilt 18
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Mz_0Ry0.in', traj, format='aims')

#    # tilt 19
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Rz_0Ry0.in', traj, format='aims')


#    # tilt 20
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Mz_0-Ry0.in', traj, format='aims')

#    # tilt 21
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_Rx0Rz_0-Ry0.in', traj, format='aims')

#    # tilt 22
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    atoms.positions += amp * atoms.get_array('Mz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_RxRyMz_RxRyMz.in', traj, format='aims')

#    # tilt 23
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    atoms.positions += amp * atoms.get_array('Rz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_RxRyRz_RxRyRz.in', traj, format='aims')

#    # tilt 24
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Mz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_RxRyMz_RxRy-Mz.in', traj, format='aims')

#    # tilt 25
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Rz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_RxRyRz_RxRy-Rz.in', traj, format='aims')

#    # tilt 26
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += -amp * atoms.get_array('Ry2')
    atoms.positions += amp * atoms.get_array('Mz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    #write(f'RP_{n}_{spglib.get_symmetry_dataset(atoms)["number"]}.in', traj, format='aims')
    write(f'tilt_structures/RP_{n}_RxRyMz_-Rx-RyMz.in', traj, format='aims')
#    # tilt 27
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += -amp * atoms.get_array('Ry2')
    atoms.positions += amp * atoms.get_array('Rz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'tilt_structures/RP_{n}_RxRyRz_-Rx-RyRz.in', traj, format='aims')
#    # tilt 28
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += -amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Mz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'tilt_structures/RP_{n}_RxRyMz_-Rx-Ry-Mz.in', traj, format='aims')
#    # tilt 29
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += -amp * atoms.get_array('Ry2')
    atoms.positions += -amp * atoms.get_array('Rz2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'tilt_structures/RP_{n}_RxRyRz_-Rx-Ry-Rz.in', traj, format='aims')
#    # tilt 30
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'tilt_structures/RP_{n}_RxRyMz_RxRy0.in', traj, format='aims')
#    # tilt 31
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Mz1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'tilt_structures/RP_{n}_RxRyMz_-Rx-Ry0.in', traj, format='aims')
#    # tilt 32
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += amp * atoms.get_array('Rx2')
    atoms.positions += amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'tilt_structures/RP_{n}_RxRyRz_RxRy0.in', traj, format='aims')
#    # tilt 33
    traj = []
    atoms = atoms_full.copy()
    atoms.positions += amp * atoms.get_array('Rx1')
    atoms.positions += amp * atoms.get_array('Ry1')
    atoms.positions += amp * atoms.get_array('Rz1')
    atoms.positions += -amp * atoms.get_array('Rx2')
    atoms.positions += -amp * atoms.get_array('Ry2')
    traj.append(atoms)

    print('tilted RP spacegroup:', spglib.get_spacegroup(atoms))
    write(f'tilt_structures/RP_{n}_RxRyRz_-RxRy0.in', traj, format='aims')

tilted RP spacegroup: I4/mmm (139)
tilted RP spacegroup: Cmce (64) MzMz
tilted RP spacegroup: P4/mbm (127) 
tilted RP spacegroup: Pbam (55)
tilted RP spacegroup: Cmce (64)
tilted RP spacegroup: P4/mbm (127)
tilted RP spacegroup: Pbam (55)
tilted RP spacegroup: P4_2/nnm (134)
tilted RP spacegroup: Cccm (66)
tilted RP spacegroup: Pnnn (48)
tilted RP spacegroup: P4_2/ncm (138)
tilted RP spacegroup: Cmce (64)
tilted RP spacegroup: Pccn (56)
tilted RP spacegroup: P2_1/c (14)
tilted RP spacegroup: P2_1/c (14)
tilted RP spacegroup: P2/c (13)
tilted RP spacegroup: P2/c (13)
tilted RP spacegroup: C2/m (12)
tilted RP spacegroup: C2/m (12)
tilted RP spacegroup: C2/m (12)
tilted RP spacegroup: C2/m (12)
tilted RP spacegroup: C2/c (15)
tilted RP spacegroup: C2/c (15)
tilted RP spacegroup: Pccn (56)
tilted RP spacegroup: Pccn (56)
tilted RP spacegroup: Pbca (61)
tilted RP spacegroup: Pbca (61)
tilted RP spacegroup: P2_1/c (14)
tilted RP spacegroup: P2_1/c (14)
tilted RP spacegroup: P2_1/c (14)
tilte